In [ ]:
import requests
import csv
import time
import re

# 打开CSV文件
f = open('try.csv',mode='a',encoding='utf-8-sig',newline='')
csv_write = csv.writer((f))
csv_write.writerow(['评论ID','用户昵称','评论内容','点赞数量','回复总数','发布时间', 'IP属地', '评论层级', '上级ID'])

# 配置
commentapi_url = "https://weibo.com/ajax/statuses/buildComments"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'Referer': 'https://weibo.com/',  
    'Cookie': 'SCF=AgxIF89gmafLBB23ychk0-sex8eo8h8UZNyxFh8u7WfjMktgE1fsaAS7NcgL5rG9N4bYSGeRMCwi8V40etajcEo.; SINAGLOBAL=3797409614845.09.1763531583040; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWpBqFZF7lBlUqdOz5kYNNN5JpX5KMhUgL.Foq41KB01hqXeo.2dJLoIEqLxK.LBo2LB.eLxKML1-qLBK-LxKML1-qLBK-LxKML1-qLBKH4UBtt; ALF=1767852037; SUB=_2A25EM8tVDeRhGeBH4lYS-CjIyTWIHXVnMUKdrDV8PUJbkNANLRDwkW1NQY9tCINw7EM1n-uFU75yv9icmiJrRX1-; ULV=1765260052375:4:1:1:8555731017538.668.1765260052373:1763723697297'
}

#清洗ip属地数据
def clean_ip_location(raw_location):
    if not raw_location:
        return "未知属地"
    cleaned_loc = re.sub(r"来自", "", raw_location).strip()
    return cleaned_loc if cleaned_loc else "未知属地"

#爬取二级评论
def get_sub_comments(parent_comment_id):
    sub_params = {
        "is_reload": 1,
        "id": parent_comment_id,  # 一级评论ID作为二级评论接口的id参数
        "is_show_bulletin": 2,
        "is_mix": 1,  # 开启混合模式获取回复
        "count": 40,
        "uid": "7762107285",
        "fetch_level": 1,  # 获取二级评论的层级
        "locale": "zh-CN"
    }
    
    try:
        response = requests.get(url=commentapi_url, headers=headers, params=sub_params)
        if response.status_code != 200:
            print(f"获取二级评论失败，状态码: {response.status_code}")
            return
        
        sub_json = response.json()
        sub_data_list = sub_json.get("data", [])
        if not sub_data_list:
            return
        
        print(f"  发现{len(sub_data_list)}条二级评论")
        
        for sub_data in sub_data_list:
            sub_id = sub_data.get("id", "无")
            sub_screen_name = sub_data.get("user", {}).get("screen_name", "未知用户")
            sub_text_raw = sub_data.get("text_raw", "无")
            sub_like_counts = sub_data.get("like_counts", 0)
            sub_total_number = sub_data.get("total_number", 0)
            sub_created_at = sub_data.get("created_at", 0)
            sub_ip_location = clean_ip_location(sub_data.get("source_allowance", sub_data.get("source", "")))
            
            csv_write.writerow([
                sub_id, sub_screen_name, sub_text_raw, sub_like_counts, 
                sub_total_number, sub_created_at, sub_ip_location, 2, parent_comment_id
            ])
        
        time.sleep(0.5)
        
    except Exception as e:
        print(f"获取二级评论异常: {str(e)}")

#爬取评论
def get_comments(max_id=None):
    params = {
        'is_reload': 1,
        'id': '5211132333656001',
        'is_show_bulletin': 2,
        'is_mix': 0,  # 0表示只获取一级评论
        'count': 40,  
        'uid': '7762107285',
        'fetch_level': 0,
        'locale': 'zh-CN'
    }

    # 分页：添加max_id参数
    if max_id:
        params['max_id'] = max_id
    
    try:
        response = requests.get(url=commentapi_url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"请求失败: {response.status_code}")
            return None
        
        json_data = response.json()
        data_list = json_data['data']

        # 检查是否有数据
        if not data_list:
            print("没有更多数据")
            return None
            
        next_max_id = json_data.get('max_id', 0)
        
        print(f"本次获取到 {len(data_list)} 条评论")

        for data in data_list:
            text_raw = data['text_raw']
            comment_id = data['id']
            created_at = data['created_at']
            like_counts = data['like_counts']
            total_number = data['total_number']
            screen_name = data['user']['screen_name']
            ip_location = clean_ip_location(data.get('source_allowance', data.get('source', '')))
            
            csv_write.writerow([
                comment_id, screen_name, text_raw, like_counts, 
                total_number, created_at, ip_location, 1, ""
            ])
            print(f"一级评论[{comment_id}]: {screen_name} - {text_raw[:20]}... (属地: {ip_location})")
            
            # 如有，调用函数爬取二级评论
            if total_number > 0:
                print(f"  开始获取二级评论：")
                get_sub_comments(comment_id)
    
        time.sleep(1)
        
        # 如果有下一页，继续获取
        if next_max_id and next_max_id != max_id and next_max_id != 0:
            print(f"获取下一页，max_id: {next_max_id}")
            get_comments(next_max_id)
        else:
            print("已获取所有数据")
            
    except Exception as e:
        print(f"发生错误: {e}")
        return None

# 开始爬取
get_comments()

f.close()

本次获取到 20 条评论
一级评论[5211132868693509]: --好柿花生 - 西贝，只有顾客是现宰的。... (属地: 北京)
  开始获取二级评论：
  发现20条二级评论
一级评论[5211136284428880]: 杨智翔Morals - 本来想阴阳几句，还是算了。一会儿点到前排... (属地: 浙江)
  开始获取二级评论：
  发现19条二级评论
一级评论[5211132496184955]: 一不小心帅爆了 - 所以顾客虐你啥了？... (属地: 山东)
  开始获取二级评论：
  发现20条二级评论
一级评论[5211132832253566]: 像雾又像风的女子 - 不要把顾客打成黑社会好吗... (属地: 广东)
  开始获取二级评论：
  发现19条二级评论
一级评论[5211132522660865]: 高兄Plus - 西贝不是知道错了，只是知道自己要死了... (属地: 山东)
  开始获取二级评论：
  发现20条二级评论
一级评论[5211132874195601]: 肉肉姐哟 - 顾客虐你啥了？？？... (属地: 四川)
  开始获取二级评论：
  发现20条二级评论
一级评论[5211133275539528]: refrain_L - 既然顾客虐你千百遍了 那你还是关门吧 别... (属地: 山东)
  开始获取二级评论：
  发现19条二级评论
一级评论[5211132749153931]: 蓝莓狸枝 - 2025最差公关[卡皮巴拉]... (属地: 浙江)
  开始获取二级评论：
  发现20条二级评论
一级评论[5211132522662196]: 唐老鸭只有两只Jio - 哈哈哈[二哈] 狼已经自爆了... (属地: 广东)
  开始获取二级评论：
  发现18条二级评论
一级评论[5211132971452044]: Metanoia_Lzq - 谁虐谁啊？！不是你西贝虐顾客么[微笑]... (属地: 黑龙江)
  开始获取二级评论：
  发现16条二级评论
一级评论[5211133430727471]: Deepdisney - 我带孩子去吃饭，一餐200起步，我怎么虐... (属地: 山东)
  开始获取二级评论：
  发现20条二级评论
一级评论[5211144962445790]: 主持人高杰 

In [ ]:
#合并数据文档
import pandas as pd
import os

csv_folder = "/Users/beslik/Documents/Files/COM5507/GroupProject/comment_data/data"  
output_file = "comments_all.csv"  

#读取所有CSV文件
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]
df_list = []

for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(
        file_path,
        encoding="utf-8", 
        na_filter=False   
    )
    df_list.append(df)
    print(f"已读取：{file}，行数：{len(df)}")

merged_df = pd.concat(df_list, ignore_index=True) 

merged_df.to_csv(
    output_file,
    index=False,  
    encoding="utf-8"
)

print(f"整合完成！输出文件：{output_file}，总行数：{len(merged_df)}")

已读取：5211132333656001.csv，行数：1221
已读取：5210068212777103.csv，行数：746
已读取：5210089700196777.csv，行数：539
整合完成！输出文件：comments_all_XB.csv，总行数：2506
